In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mdtraj as md
from scipy.spatial import cKDTree
import sys
from time import time
from jinja2 import Template
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import Akima1DInterpolator
from scipy.signal import find_peaks
from scipy.optimize import brentq
from scipy.optimize import curve_fit
import matplotlib.lines as mlines
import matplotlib as mpl

try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd $workdir

In [ ]:
def avgConc(N,n_range,atom,method):
    Yup = []
    Ylo = []
    for n in n_range:
        params = pd.read_pickle(method+'/{:d}/{:d}.p'.format(n,N))
        data, z = params['data'], params['z']
        if type(data[atom]['upper']) == dict:
            yup,ylo = data[atom]['upper']['conc'],data[atom]['lower']['conc']
        else:
            yup,ylo = data[atom]['upper'],data[atom]['lower']
        Yup.append(yup)
        Ylo.append(ylo)
    yup = np.mean(Yup,axis=0)
    ylo = np.mean(Ylo,axis=0)
    y, yerr = .5*(yup+ylo), .5*(yup-ylo)
    peaks, _ = find_peaks(y,height=1,width=1)
    localmin = y[peaks[0]:peaks[1]].argmin()+peaks[0]
    midz = (z[peaks[-1]] - z[peaks[0]])/2 
    conc = y[np.logical_and(z>midz-1,z<midz+2)].mean()
    return z, yup, ylo, localmin, conc

def pltConc(ax,atom,Ns,method):
    surfconc = []
    excess = []
    for i,N in enumerate(Ns):
        z, yup, ylo, localmin, _ = avgConc(N,range(10),atom,method)
        y, yerr = .5*(yup+ylo), .5*(yup-ylo)
        _, __, ___, ____, conc = avgConc(N,range(10),atom,'gds')
        
        ax.plot(z,y,color=plt.cm.tab10(i),label='{:.1f}'.format(conc))
        ax.fill_between(z,y-yerr,y+yerr,alpha=.5,color=plt.cm.tab10(i))
        
        intup = 6.022/10*np.trapz(yup[:localmin],z[:localmin])
        intlo = 6.022/10*np.trapz(ylo[:localmin],z[:localmin])
        surfconc.append([conc,.5*(intup+intlo),np.abs(intup-intlo)/2.])
        
        excint = np.logical_and(z>0,z<2.5)
        excup = 6.022/10*(np.trapz(yup[z<=0],z[z<=0])+np.trapz(yup[excint]-conc,z[excint]))
        exclo = 6.022/10*(np.trapz(ylo[z<=0],z[z<=0])+np.trapz(ylo[excint]-conc,z[excint]))
        excess.append([conc,.5*(excup+exclo),np.abs(excup-exclo)/2.])
    return np.array(surfconc).reshape(-1,3), np.array(excess).reshape(-1,3) #, np.array(localmins).reshape(-1,3)

def pltConcNorm(ax,atom,Ns,method,color):
    surfconc = []
    excess = []
    #localmins = []
    for i,N in enumerate(Ns):
        z, yup, ylo, localmin, _ = avgConc(N,range(10),atom,method)
        y, yerr = .5*(yup+ylo), .5*(yup-ylo)
        _, __, ___, ____, conc = avgConc(N,range(10),atom,'gds')
        ax.plot(z*10,y/conc,color=color,label=atom[0],lw=2)
        ax.fill_between(z*10,(y-yerr)/conc,(y+yerr)/conc,alpha=.5,color=color)
                         
def pltDist(ax,atom,Ns,method,layer,color):
    for i,N in enumerate(Ns):
        theta = pd.read_pickle(method+'/0/{:d}.p'.format(N))['theta']/180*np.pi
        pu = np.zeros(theta.size)
        pl = np.zeros(theta.size)
        for n in range(10):
            params = pd.read_pickle(method+'/{:d}/{:d}.p'.format(n,N))
            pu += params['data'][atom]['upper']['theta'][layer]
        pu = pu / np.trapz(pu,theta) / np.sin(theta)
        for n in range(10):
            params = pd.read_pickle(method+'/{:d}/{:d}.p'.format(n,N))
            pl += params['data'][atom]['lower']['theta'][layer]
        pl = pl / np.trapz(pl,theta) / np.sin(theta)
        p = .5*(pu+pl)
        dp = .5*(pu-pl)
        ax.plot(theta*180/np.pi,p,c=color) 
        ax.fill_between(theta*180/np.pi,p+dp,p-dp,color=color,alpha=.2)
        
def pltCosine(ax,atom,Ns,method):
    surfconc = []
    for i,N in enumerate(Ns):
        z, yup, ylo, localmin, _ = avgConc(N,range(10),atom,method)
        y, yerr = .5*(yup+ylo), .5*(yup-ylo)
        _, __, ___, ____, conc = avgConc(N,range(10),atom,'gds')
        Yupcos = []
        Ylocos = []
        for n in range(10):
            params = pd.read_pickle(method+'/{:d}/{:d}.p'.format(n,N))
            data = params['data'][atom]
            yupcos,ylocos = data['upper']['cosine'],data['lower']['cosine']
            Yupcos.append(yupcos)
            Ylocos.append(ylocos)
        yupcos = np.mean(Yupcos,axis=0)
        ylocos = np.mean(Ylocos,axis=0)
        
        ycos = .5*(yupcos+ylocos)
        yerrcos = .5*np.abs(yupcos-ylocos)
        ax.plot(z,ycos,color=plt.cm.tab10(i),label='{:.1f}'.format(conc))
        ax.fill_between(z,ycos-yerrcos,ycos+yerrcos,alpha=.5,color=plt.cm.tab10(i))
        
        intup = np.trapz(yupcos[z<2],z[z<2])
        intlo = np.trapz(ylocos[z<2],z[z<2])
        surfconc.append([conc,6.022/10*.5*(intup+intlo),np.abs(intup-intlo)/2])
    return np.array(surfconc).reshape(-1,3)

def pltCosineNorm(ax,atom,Ns,method):
    surfconc = []
    for i,N in enumerate(Ns):
        z, yup, ylo, localmin, _ = avgConc(N,range(10),atom,method)
        y, yerr = .5*(yup+ylo), .5*(yup-ylo)
        _, __, ___, ____, conc = avgConc(N,range(10),atom,'gds')
        Yupcos = []
        Ylocos = []
        for n in range(10):
            params = pd.read_pickle(method+'/{:d}/{:d}.p'.format(n,N))
            data = params['data'][atom]
            yupcos,ylocos = data['upper']['cosine'],data['lower']['cosine']
            Yupcos.append(yupcos)
            Ylocos.append(ylocos)
        yupcos = np.mean(Yupcos,axis=0)/yup
        ylocos = np.mean(Ylocos,axis=0)/ylo
        
        ycos = .5*(yupcos+ylocos)
        yerrcos = .5*np.abs(yupcos-ylocos)
        ax.plot(z,ycos,color=plt.cm.tab10(i),label='{:.1f}'.format(conc))
        ax.fill_between(z,ycos-yerrcos,ycos+yerrcos,alpha=.5,color=plt.cm.tab10(i))
        
def pltDang(ax,Ns,method,color=None,l=''):
    for i,N in enumerate(Ns):
        if color==None:
            color=i
        theta = pd.read_pickle(method+'/0/{0}.p'.format(N))['theta']/180*np.pi
        pu = np.zeros(theta.size)
        pl = np.zeros(theta.size)
        for n in range(10):
            params = pd.read_pickle(method+'/{0}/{1}.p'.format(n,N))
            pu += params['data']['O']['upper']['theta']
            pl += params['data']['O']['lower']['theta']
        pu = pu / np.trapz(pu,theta) / np.sin(theta)
        pl = pl / np.trapz(pl,theta) / np.sin(theta)
        p = .5*(pu+pl)
        dp = .5*np.abs(pu-pl)
        conc = 0
        
        par,V = np.polyfit(theta[theta<150/180*np.pi], np.log(p[theta<150/180*np.pi]), 1, cov=True)
        
        ax.fill_between(theta*180/np.pi,p+dp,p-dp,color=plt.cm.tab10(color),alpha=.2)
        
        ax.plot(theta*180/np.pi,np.exp(par[1]+par[0]*theta),lw=1,color=plt.cm.tab10(color),ls=':')
        
        if 'C2' in params['data']:
            _, __, ___, ____, conc = avgConc(N,range(10),'C2','gds')
            ax.plot(theta*180/np.pi,p,c=plt.cm.tab10(color),lw=2,
                    label=l+r'$c_s={:.1f}$ M, $\psi_E={:.0f}˚$'.format(conc,-180/np.pi/par[0]))
        else:
            ax.plot(theta*180/np.pi,p,c=plt.cm.tab10(color),lw=2,
                    label=l+r'$\psi_E={:.0f}˚$'.format(-180/np.pi/par[0]))
        color=None 
        
def pltDangLog(ax,Ns,method,color=None,l=''):
    for i,N in enumerate(Ns):
        if color==None:
            color=i
        theta = pd.read_pickle(method+'/0/{0}.p'.format(N))['theta']
        pu = np.zeros(theta.size)
        pl = np.zeros(theta.size)
        for n in range(10):
            params = pd.read_pickle(method+'/{0}/{1}.p'.format(n,N))
            pu += params['data']['O']['upper']['theta']
        pu = pu / np.trapz(pu,theta) / np.sin(theta*np.pi/180.)
        for n in range(10):
            params = pd.read_pickle(method+'/{0}/{1}.p'.format(n,N))
            pl += params['data']['O']['lower']['theta']
        pl = pl / np.trapz(pl,theta) / np.sin(theta*np.pi/180.)
        p = .5*(pu+pl)
        dp = .5*np.abs(pu-pl)
        
        par,V = np.polyfit(theta[theta<150], np.log(p[theta<150]), 1, cov=True)
        
        ax.plot(theta,np.log(p),c=plt.cm.tab10(color),lw=2,label=l+r'$\psi_E={:.0f}˚$'.format(-1/par[0]))
        ax.fill_between(theta,np.log(p+dp),np.log(p-dp),color=plt.cm.tab10(color),alpha=.2)
        
        ax.plot(theta,par[1]+par[0]*theta,lw=1,color=plt.cm.tab10(color),ls=':')
        
def pltDang(ax,Ns,method,color=None,l=''):
    for i,N in enumerate(Ns):
        if color==None:
            color=i
        theta = pd.read_pickle(method+'/0/{0}.p'.format(N))['theta']/180*np.pi
        pu = np.zeros(theta.size)
        pl = np.zeros(theta.size)
        for n in range(10):
            params = pd.read_pickle(method+'/{0}/{1}.p'.format(n,N))
            pu += params['data']['O']['upper']['theta']
            pl += params['data']['O']['lower']['theta']
        pu = pu / np.trapz(pu,theta) / np.sin(theta)
        pl = pl / np.trapz(pl,theta) / np.sin(theta)
        p = .5*(pu+pl)
        dp = .5*np.abs(pu-pl)
        conc = 0
        
        func = lambda x,a : np.log(a*a + 1) - np.log(a*a*(np.exp(-np.pi/a)+1)) - x / a  
        
        popt, pcov = curve_fit(func, theta, np.log(p), sigma=dp/p)
        
        ax.fill_between(theta*180/np.pi,p+dp,p-dp,color=plt.cm.tab10(color),alpha=.2,lw=0)
        
        #ax.plot(theta*180/np.pi,np.exp(func(theta,popt[0])),lw=1,color=plt.cm.tab10(color),ls=':')
        
        if 'C2' in params['data']:
            _, __, ___, ____, conc = avgConc(N,range(10),'C2','gds')
            ax.plot(theta*180/np.pi,p,c=plt.cm.tab10(color),lw=2,
                    label=l+r'{:.1f}'.format(conc))
            print(l+r'$c_s={:.1f}$ M, $\psi_E={:.0f}˚$'.format(conc,popt[0]*180/np.pi))
        else:
            ax.plot(theta*180/np.pi,p,c=plt.cm.tab10(color),lw=2,label=l)
            print(l+r'$\psi_E={:.0f}˚$'.format(popt[0]*180/np.pi))
        color=None
        
def pltDangLog(ax,Ns,method,color=None,l=''):
    for i,N in enumerate(Ns):
        if color==None:
            color=i
        theta = pd.read_pickle(method+'/0/{0}.p'.format(N))['theta']/180*np.pi
        pu = np.zeros(theta.size)
        pl = np.zeros(theta.size)
        for n in range(10):
            params = pd.read_pickle(method+'/{0}/{1}.p'.format(n,N))
            pu += params['data']['O']['upper']['theta']
            pl += params['data']['O']['lower']['theta']
        pu = pu / np.trapz(pu,theta) / np.sin(theta)
        pl = pl / np.trapz(pl,theta) / np.sin(theta)
        p = .5*(pu+pl)
        dp = .5*np.abs(pu-pl)
        conc = 0
        
        func = lambda x,a : np.log(a*a + 1) - np.log(a*a*(np.exp(-np.pi/a)+1)) - x / a  
        
        popt, pcov = curve_fit(func, theta, np.log(p), sigma=dp/p)
        
        ax.fill_between(theta*180/np.pi,np.log(p)+dp/p,np.log(p)-dp/p,
                        color=plt.cm.tab10(color),alpha=.2,lw=0)
        
        ax.plot(theta*180/np.pi,func(theta,popt[0]),lw=1,color=plt.cm.tab10(color),ls=':')
        
        if 'C2' in params['data']:
            _, __, ___, ____, conc = avgConc(N,range(10),'C2','gds')
            ax.plot(theta*180/np.pi,np.log(p),c=plt.cm.tab10(color),lw=2,
                    label=l+r'{:.1f}'.format(conc))
            print(l+r'$c_s={:.1f}$ M, $\psi_E={:.0f}˚$'.format(conc,popt[0]*180/np.pi))
        else:
            ax.plot(theta*180/np.pi,np.log(p),c=plt.cm.tab10(color),lw=2,label=l)
            print(l+r'$\psi_E={:.0f}˚$'.format(popt[0]*180/np.pi))
        color=None
        
def plotLog(ax,filename,color,l):
    a,b = np.loadtxt(filename,unpack=True)
    a = a/180*np.pi
    func = lambda x,a : np.log(a*a + 1) - np.log(a*a*(np.exp(-np.pi/a)+1)) - x / a  
    popt, pcov = curve_fit(func, a, np.log(b))
    ax.plot(a*180/np.pi,np.log(b),c=plt.cm.tab10(color),lw=2,label=l+r', $\psi_E={:.0f}˚$'.format(popt[0]*180/np.pi))
    ax.plot(a*180/np.pi,func(a,popt[0]), lw=1,c=plt.cm.tab10(color),ls=':')
    
def plotExp(ax,filename,color,l):
    a,b = np.loadtxt(filename,unpack=True)
    a = a/180*np.pi
    func = lambda x,a : np.log(a*a + 1) - np.log(a*a*(np.exp(-np.pi/a)+1)) - x / a  
    popt, pcov = curve_fit(func, a, np.log(b))
    ax.plot(a*180/np.pi,b,c=plt.cm.tab10(color),lw=2,label=l+r', $\psi_E={:.0f}˚$'.format(popt[0]*180/np.pi))
    ax.plot(a*180/np.pi,np.exp(func(a,popt[0])), lw=1,c=plt.cm.tab10(color),ls=':')
    
def pltFrac(ax,Ns,method,color=None,l=''):
    f = np.empty(0)
    ferr = np.empty(0)
    cs = np.empty(0)
    for i,N in enumerate(Ns):
        print(N)
        frac = np.empty(0)
        for n in range(10):
            params = pd.read_pickle(method+'/{0}/{1}.p'.format(n,N))
            a = params['data']['O']['upper']['ndang']
            b = params['data']['O']['upper']['all']
            frac = np.append(frac, a/b)
            a = params['data']['O']['lower']['ndang']
            b = params['data']['O']['lower']['all']
            frac = np.append(frac, a/b)
            
            print(N,n,params['level']*2)
            
        if 'C2' in params['data']:
            _, __, ___, ____, conc = avgConc(N,range(10),'C2','gds')
        else:
            conc = 0
        cs = np.append(cs,conc)
        f = np.append(f,np.mean(frac)*100)
        ferr = np.append(ferr,np.std(frac)*100)
    
    print(f,ferr)
        
    ax.errorbar(cs,f,ferr,marker='o',color=color,
        markeredgecolor=None,markersize=10,elinewidth=2,capsize=4,
        lw=0,capthick=2.,alpha=1)
    
def pltNumDang(ax,Ns,method,color=None,l=''):
    f = np.empty(0)
    ferr = np.empty(0)
    cs = np.empty(0)
    for i,N in enumerate(Ns):
        print(N)
        frac = np.empty(0)
        for n in range(10):
            params = pd.read_pickle(method+'/{0}/{1}.p'.format(n,N))
            a = params['data']['O']['upper']['ndang']
            frac = np.append(frac, a)
            a = params['data']['O']['lower']['ndang']
            frac = np.append(frac, a)
            
        if 'C2' in params['data']:
            _, __, ___, ____, conc = avgConc(N,range(10),'C2','gds')
        else:
            conc = 0
        cs = np.append(cs,conc)
        f = np.append(f,np.mean(frac))
        ferr = np.append(ferr,np.std(frac))
        
    ax.errorbar(cs,f,ferr,marker='o',color=color,
        markeredgecolor=None,markersize=10,elinewidth=2,capsize=4,
        lw=0,capthick=2.,alpha=1)
    
def pltAveragePsi(ax,Ns,method,color=None,l=''):
    psi = np.empty(0)
    psierr = np.empty(0)
    cs = np.empty(0)
    for i,N in enumerate(Ns):
        theta = pd.read_pickle(method+'/0/{0}.p'.format(N))['theta']/180*np.pi
        pu = np.zeros(theta.size)
        pl = np.zeros(theta.size)
        for n in range(10):
            params = pd.read_pickle(method+'/{0}/{1}.p'.format(n,N))
            pu += params['data']['O']['upper']['theta']
            pl += params['data']['O']['lower']['theta']
        pu = pu / np.trapz(pu,theta) / np.sin(theta)
        pl = pl / np.trapz(pl,theta) / np.sin(theta)
        
        psiu = np.trapz(theta*pu*np.sin(theta),theta)
        psil = np.trapz(theta*pl*np.sin(theta),theta)
        
        if 'C2' in params['data']:
            _, __, ___, ____, conc = avgConc(N,range(10),'C2','gds')
        else:
            conc = 0
        cs = np.append(cs,conc)
        psi = np.append(psi,.5*(psiu+psil))
        psierr = np.append(psierr,.5*np.abs(psiu-psil))
        
    print(psi*180/np.pi,psierr*180/np.pi)
        
    ax.errorbar(cs,psi*180/np.pi,psierr*180/np.pi,marker='o',color=color,
        markeredgecolor=None,markersize=10,elinewidth=2,capsize=4,
        lw=0,capthick=2.,alpha=1)

In [ ]:
for N,c in zip(['water',447,596,894,1192,150,300,450,600,800],[0,1.5,2,3,4,.8,1.7,2.6,3.5,4.7]):
    avg = np.empty(0)
    d1 = np.empty(0)
    d2 = np.empty(0)
    for n in range(1):
        params = pd.read_pickle('gds/{0}/{1}.p'.format(n,N))
        avg = np.append(avg, params['level'])
        d1 = np.append(d1, params['thickness'][0])
        d2 = np.append(d1, params['thickness'][1])
    print(N,c,avg.mean()*2,avg.std()*2,.5*(d1.mean()+d2.mean()),np.abs(d2.mean()-d1.mean()))

In [ ]:
plt.rcParams.update({'font.size': 15,'xtick.major.pad':3,'ytick.major.pad':3,'figure.dpi':150,
                     'xtick.major.size':6,'ytick.major.size':6,'legend.fontsize':15,
                     'xtick.direction':'out','ytick.direction':'out','axes.labelsize':15,
                     'axes.linewidth':1.2,'xtick.labelsize':15,'ytick.labelsize':15,
                     'xtick.major.width':1.2, 'ytick.major.width':1.2})

In [ ]:
plt.rcParams.update({'figure.figsize': [10, 10], 'figure.dpi':70})
fig, ((ax1, ax2),(ax3, ax4),(ax5, ax6)) = plt.subplots(3, 2, sharex=True, sharey=False)

for curve,color,l in zip(['yellow','green','red'],[8,2,3],['POLI2VS','TIP3P','TIP4P/ice']):
    plotLog(ax2,'Sun-et-al/{0}.csv'.format(curve),color,l)
    
for curve,color,l in zip(['yellow','green','red'],[8,2,3],['POLI2VS','TIP3P','TIP4P/ice']):
    plotExp(ax1,'Sun-et-al/{0}.csv'.format(curve),color,l)  
    
for curve,color,l in zip(['redSI','blueSI'],[3,0],['POLI2VS, WC','POLI2VS, GDS']):
    plotLog(ax4,'Sun-et-al/{0}.csv'.format(curve),color,l)
    
for curve,color,l in zip(['redSI','blueSI'],[3,0],['POLI2VS, WC','POLI2VS, GDS']):
    plotExp(ax3,'Sun-et-al/{0}.csv'.format(curve),color,l)
    
pltDangLog(ax6,['water'],'wcis',3,l='SPC/E, WC, ')
pltDangLog(ax6,['water'],'gds',0,l='SPC/E, GDS, ')

pltDang(ax5,['water'],'wcis',3,l='SPC/E, WC, ')
pltDang(ax5,['water'],'gds',0,l='SPC/E, GDS, ')
    
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
ax4.yaxis.tick_right()
ax4.yaxis.set_label_position("right")
ax6.yaxis.tick_right()
ax6.yaxis.set_label_position("right")

ax2.set_ylabel(r'Log Probability')
ax4.set_ylabel(r'Log Probability')
ax6.set_ylabel(r'Log Probability')

ax5.set_xlabel(r'Dangling OH tilt angle, $\psi$ (˚)')
ax6.set_xlabel(r'Dangling OH tilt angle, $\psi$ (˚)')

for ax in [ax1,ax3,ax5]:
    ax.set_ylabel(r'Probability')
    ax.legend(handletextpad=0.5,markerfirst=False,frameon=False,handlelength=1,loc='center right')
    
    
for ax,l in zip(fig.axes,['a','b','c','d','e','f']):
    ax.annotate(l,xy=(0.9,.85), fontsize=22, xycoords='axes fraction')

plt.tight_layout(w_pad=.5,h_pad=.5)

plt.savefig('figs/comparison.pdf')

In [ ]:
plt.rcParams.update({'figure.figsize': [13, 7]})
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=False, sharey=False)
pltConcNorm(ax1,'S1',[1192],'gds',plt.cm.tab10(8))
pltConcNorm(ax1,'C2',[1192],'gds',plt.cm.tab10(7))
pltConcNorm(ax1,'N3',[1192],'gds',plt.cm.tab10(0))
pltConcNorm(ax2,'S1',[600],'gds',plt.cm.tab10(8))
pltConcNorm(ax2,'C2',[600],'gds',plt.cm.tab10(7))
pltConcNorm(ax2,'N3',[600],'gds',plt.cm.tab10(0))
pltConcNorm(ax3,'S1',[1192],'wcis',plt.cm.tab10(8))
pltConcNorm(ax3,'C2',[1192],'wcis',plt.cm.tab10(7))
pltConcNorm(ax3,'N3',[1192],'wcis',plt.cm.tab10(0))
pltConcNorm(ax4,'S1',[600],'wcis',plt.cm.tab10(8))
pltConcNorm(ax4,'C2',[600],'wcis',plt.cm.tab10(7))
pltConcNorm(ax4,'N3',[600],'wcis',plt.cm.tab10(0))

for ax,title in zip(f.axes,['a','b','c','d']):
    ax.set_ylabel('Local Concentration / $c_s$')  
    ax.set_xlim(-4,14)
    ax.legend(loc='center right',markerfirst=False)
    ax.axvspan(-1, 1, color=plt.cm.tab10(2), alpha=0.1, ec=None)
    ax.axvspan(1, 3, color=plt.cm.tab10(3), alpha=0.1, ec=None)
    ax.axvspan(3, 5, color=plt.cm.tab10(4), alpha=0.1, ec=None)
    ax.axvspan(5, 7, color=plt.cm.tab10(8), alpha=0.1, ec=None)
    ax.axvspan(7, 9, color=plt.cm.tab10(7), alpha=0.1, ec=None)
    leg = ax.legend(frameon=False,loc='upper right',handlelength=1.2,markerfirst=False)
    leg.set_title(title, prop = {'size':22})

for ax in [ax1,ax2]:   
    ax.set_ylim(-.07,2.5)
    ax.set_yticks(np.arange(0,2.1,1))
    ax.set_xticklabels(['']*8)
    

for ax in [ax3,ax4]:
    ax.set_ylim(-.12,3.5)
    ax.set_yticks(np.arange(0,3.6,1))
    ax.set_yticklabels(['{:1g}'.format(i) for i in np.arange(0,4.1,1)])
    ax.set_xlabel(r'Distance from Interface, $z$ (Å)')    

ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
ax4.yaxis.tick_right()
ax4.yaxis.set_label_position("right")

#ax4.vlines(x=63,ymin=0,ymax=3)
#ax4.vlines(x=40,ymin=0,ymax=3)
#ax4.vlines(x=85,ymin=0,ymax=3)
#ax4.hlines(y=1.3,xmin=30,xmax=120)

plt.tight_layout(w_pad=.5, h_pad=0.5)
plt.savefig('figs/fig3.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [13, 4]})
f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False)
pltConcNorm(ax1,'S1',[1192],'wcis',plt.cm.tab10(8))
pltConcNorm(ax1,'C2',[1192],'wcis',plt.cm.tab10(7))
pltConcNorm(ax1,'N3',[1192],'wcis',plt.cm.tab10(0))
pltConcNorm(ax2,'S1',[600],'wcis',plt.cm.tab10(8))
pltConcNorm(ax2,'C2',[600],'wcis',plt.cm.tab10(7))
pltConcNorm(ax2,'N3',[600],'wcis',plt.cm.tab10(0))

for ax,title in zip(f.axes,['a','b']):
    ax.set_ylabel('Local Concentration / $c_s$')  
    ax.set_xlim(-4,14)
    ax.legend(loc='center right',markerfirst=False)
    ax.axvspan(-1, 1, color=plt.cm.tab10(2), alpha=0.1, ec=None)
    ax.axvspan(1, 3, color=plt.cm.tab10(3), alpha=0.1, ec=None)
    ax.axvspan(3, 5, color=plt.cm.tab10(4), alpha=0.1, ec=None)
    ax.axvspan(5, 7, color=plt.cm.tab10(8), alpha=0.1, ec=None)
    ax.axvspan(7, 9, color=plt.cm.tab10(7), alpha=0.1, ec=None)
    leg = ax.legend(frameon=False,loc='upper right',handlelength=1.2,markerfirst=False)

for ax in [ax1,ax2]:   
    ax.set_ylim(-.12,3.5)
    ax.set_yticks(np.arange(0,3.6,1))
    ax.set_yticklabels(['{:1g}'.format(i) for i in np.arange(0,4.1,1)])
    ax.set_xlabel(r'Distance from Instantaneous Interface, $z$ (Å)')
    

ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")

#ax4.vlines(x=63,ymin=0,ymax=3)
#ax4.vlines(x=40,ymin=0,ymax=3)
#ax4.vlines(x=85,ymin=0,ymax=3)
#ax4.hlines(y=1.3,xmin=30,xmax=120)

plt.tight_layout(w_pad=2, h_pad=0.5)
plt.savefig('figs/fig3_wcis.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [13, 4]})
f, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False)
pltConcNorm(ax1,'S1',[1192],'gds',plt.cm.tab10(8))
pltConcNorm(ax1,'C2',[1192],'gds',plt.cm.tab10(7))
pltConcNorm(ax1,'N3',[1192],'gds',plt.cm.tab10(0))
pltConcNorm(ax2,'S1',[600],'gds',plt.cm.tab10(8))
pltConcNorm(ax2,'C2',[600],'gds',plt.cm.tab10(7))
pltConcNorm(ax2,'N3',[600],'gds',plt.cm.tab10(0))

for ax,title in zip(f.axes,['a','b']):
    ax.set_ylabel('Local Concentration / $c_s$')  
    ax.set_xlim(-4,14)
    ax.legend(loc='center right',markerfirst=False)
    ax.axvspan(-1, 1, color=plt.cm.tab10(2), alpha=0.1, ec=None)
    ax.axvspan(1, 3, color=plt.cm.tab10(3), alpha=0.1, ec=None)
    ax.axvspan(3, 5, color=plt.cm.tab10(4), alpha=0.1, ec=None)
    ax.axvspan(5, 7, color=plt.cm.tab10(8), alpha=0.1, ec=None)
    ax.axvspan(7, 9, color=plt.cm.tab10(7), alpha=0.1, ec=None)
    leg = ax.legend(frameon=False,loc='upper right',handlelength=1.2,markerfirst=False)

for ax in [ax1,ax2]:   
    ax.set_ylim(-.12,3.5)
    ax.set_yticks(np.arange(0,3.6,1))
    ax.set_yticklabels(['{:1g}'.format(i) for i in np.arange(0,4.1,1)])
    ax.set_xlabel(r'Distance from Gibbs Dividing Surface, $z$ (Å)')
    

ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")

#ax4.vlines(x=63,ymin=0,ymax=3)
#ax4.vlines(x=40,ymin=0,ymax=3)
#ax4.vlines(x=85,ymin=0,ymax=3)
#ax4.hlines(y=1.3,xmin=30,xmax=120)

plt.tight_layout(w_pad=2, h_pad=0.5)
plt.savefig('figs/fig3_gds.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [6, 8]})
f, (ax1, ax2) = plt.subplots(2, 1, sharex=False, sharey=False)
pltConcNorm(ax1,'S1',[1192],'gds',plt.cm.tab10(8))
pltConcNorm(ax1,'C2',[1192],'gds',plt.cm.tab10(7))
pltConcNorm(ax1,'N3',[1192],'gds',plt.cm.tab10(0))
pltConcNorm(ax2,'S1',[1192],'wcis',plt.cm.tab10(8))
pltConcNorm(ax2,'C2',[1192],'wcis',plt.cm.tab10(7))
pltConcNorm(ax2,'N3',[1192],'wcis',plt.cm.tab10(0))

for ax,title in zip(f.axes,['a','b']):
    ax.set_ylabel('Local Concentration / $c_s$')  
    ax.set_xlim(-4,14)
    ax.legend(loc='center right',markerfirst=False)
    ax.axvspan(-1, 1, color=plt.cm.tab10(2), alpha=0.1, ec=None)
    ax.axvspan(1, 3, color=plt.cm.tab10(3), alpha=0.1, ec=None)
    ax.axvspan(3, 5, color=plt.cm.tab10(4), alpha=0.1, ec=None)
    ax.axvspan(5, 7, color=plt.cm.tab10(8), alpha=0.1, ec=None)
    ax.axvspan(7, 9, color=plt.cm.tab10(7), alpha=0.1, ec=None)
    leg = ax.legend(frameon=False,loc='upper right',handlelength=1.2,markerfirst=False)

for ax in [ax1,ax2]:   
    ax.set_ylim(-.07,2.2)
    ax.set_yticks(np.arange(0,2.1,1))
    ax.set_yticklabels(['{:1g}'.format(i) for i in np.arange(0,4.1,1)])

ax1.set_xlabel(r'Distance from Gibbs Dividing Surface, $z$ (Å)')
ax2.set_xlabel(r'Distance from Instantaneous Interface, $z$ (Å)')

#ax4.vlines(x=63,ymin=0,ymax=3)
#ax4.vlines(x=40,ymin=0,ymax=3)
#ax4.vlines(x=85,ymin=0,ymax=3)
#ax4.hlines(y=1.3,xmin=30,xmax=120)

plt.tight_layout(w_pad=0, h_pad=2)
plt.savefig('figs/fig3_gdsvswcis.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [13, 7]})
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=False, sharey=False)
pltConcNorm(ax1,'S1',[596],'gds',plt.cm.tab10(8))
pltConcNorm(ax1,'C2',[596],'gds',plt.cm.tab10(7))
pltConcNorm(ax1,'N3',[596],'gds',plt.cm.tab10(0))
pltConcNorm(ax2,'S1',[300],'gds',plt.cm.tab10(8))
pltConcNorm(ax2,'C2',[300],'gds',plt.cm.tab10(7))
pltConcNorm(ax2,'N3',[300],'gds',plt.cm.tab10(0))
pltConcNorm(ax3,'S1',[596],'wcis',plt.cm.tab10(8))
pltConcNorm(ax3,'C2',[596],'wcis',plt.cm.tab10(7))
pltConcNorm(ax3,'N3',[596],'wcis',plt.cm.tab10(0))
pltConcNorm(ax4,'S1',[300],'wcis',plt.cm.tab10(8))
pltConcNorm(ax4,'C2',[300],'wcis',plt.cm.tab10(7))
pltConcNorm(ax4,'N3',[300],'wcis',plt.cm.tab10(0))

for ax,title in zip(f.axes,['a','b','c','d']):
    ax.set_ylabel('Local Concentration / $c_s$')  
    ax.set_xlim(-4,14)
    ax.legend(loc='center right',markerfirst=False)
    ax.axvspan(-1, 1, color=plt.cm.tab10(2), alpha=0.1, ec=None)
    ax.axvspan(1, 3, color=plt.cm.tab10(3), alpha=0.1, ec=None)
    ax.axvspan(3, 5, color=plt.cm.tab10(4), alpha=0.1, ec=None)
    ax.axvspan(5, 7, color=plt.cm.tab10(8), alpha=0.1, ec=None)
    ax.axvspan(7, 9, color=plt.cm.tab10(7), alpha=0.1, ec=None)
    leg = ax.legend(frameon=False,loc='upper right',handlelength=1.2,markerfirst=False)
    leg.set_title(title, prop = {'size':22})

for ax in [ax1,ax2]:   
    ax.set_ylim(-.07,2.5)
    ax.set_yticks(np.arange(0,2.1,1))
    ax.set_xticklabels(['']*8)
    

for ax in [ax3,ax4]:
    ax.set_ylim(-.12,4)
    ax.set_yticks(np.arange(0,4.1,1))
    ax.set_yticklabels(['{:1g}'.format(i) for i in np.arange(0,4.1,1)])
    ax.set_xlabel(r'Distance from Interface, $z$ (Å)')    

ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
ax4.yaxis.tick_right()
ax4.yaxis.set_label_position("right")

#ax4.vlines(x=63,ymin=0,ymax=3)
#ax4.vlines(x=40,ymin=0,ymax=3)
#ax4.vlines(x=85,ymin=0,ymax=3)
#ax4.hlines(y=1.3,xmin=30,xmax=120)

plt.tight_layout(w_pad=.5, h_pad=0.5)
plt.savefig('figs/fig3alt.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [6.5, 7]})
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=False)

for i,j in zip([0,1,2,3,4],[2,3,4,8,7]):
    pltDist(ax1,'C2',[1192],'wcis',i,plt.cm.tab10(j))
    pltDist(ax2,'C2',[600],'wcis',i,plt.cm.tab10(j))
    
l1 = mlines.Line2D([], [], color=plt.cm.tab10(2), label='-1 Å$<z<$1 Å',lw=3,alpha=.6)
l2 = mlines.Line2D([], [], color=plt.cm.tab10(3), label='1 Å$<z<$3 Å',lw=3,alpha=.6)
l3 = mlines.Line2D([], [], color=plt.cm.tab10(4), label='3 Å$<z<$5 Å',lw=3,alpha=.6)
l4 = mlines.Line2D([], [], color=plt.cm.tab10(8), label='5 Å$<z<$7 Å',lw=3,alpha=.6)
l5 = mlines.Line2D([], [], color=plt.cm.tab10(7), label='7 Å$<z<$9 Å',lw=3,alpha=.6)
ax2.set_xlabel(r'SCN$^-$ Tilt Angle, $\theta$ (˚)')
for ax in [ax1,ax2]:
    ax.legend(handles=[l1,l2,l3,l4,l5],handlelength=1.5,framealpha=1,labelspacing=.3,borderpad=0.5,
                    handletextpad=0.5,loc='upper right',frameon=False,markerfirst=False)
    ax.get_legend()._legend_box.align = 'right'
    ax.set_yticks(np.arange(0,2.1,1))
    ax.set_ylim(-.14,2.5)
    ax.set_xlim(0,180)
    ax.set_xticks(np.arange(0,181,30))
    ax.set_ylabel(r'Probability, $P(\theta)$')
ax1.annotate(r'a',xy=(0.05,.88), fontsize=22, xycoords='axes fraction')
ax2.annotate(r'b',xy=(0.05,.88), fontsize=22, xycoords='axes fraction') 

#ax2.vlines(x=63,ymin=0,ymax=3)
#ax2.vlines(x=45.5,ymin=0,ymax=1.5)
#ax2.vlines(x=80.5,ymin=0,ymax=1.5)
    
plt.tight_layout(h_pad=0.35)
plt.savefig('figs/fig4.pdf')

In [ ]:
plt.rcParams.update({'figure.figsize': [6.5, 7]})
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=False)

for i,j in zip([0,1,2,3,4],[2,3,4,8,7]):
    pltDist(ax1,'C2',[1192],'gds',i,plt.cm.tab10(j))
    pltDist(ax2,'C2',[600],'gds',i,plt.cm.tab10(j))
    
l1 = mlines.Line2D([], [], color=plt.cm.tab10(2), label='-1 Å$<z<$1 Å',lw=3,alpha=.6)
l2 = mlines.Line2D([], [], color=plt.cm.tab10(3), label='1 Å$<z<$3 Å',lw=3,alpha=.6)
l3 = mlines.Line2D([], [], color=plt.cm.tab10(4), label='3 Å$<z<$5 Å',lw=3,alpha=.6)
l4 = mlines.Line2D([], [], color=plt.cm.tab10(8), label='5 Å$<z<$7 Å',lw=3,alpha=.6)
l5 = mlines.Line2D([], [], color=plt.cm.tab10(7), label='7 Å$<z<$9 Å',lw=3,alpha=.6)
ax2.set_xlabel(r'SCN$^-$ Tilt Angle, $\theta$ (˚)')
for ax in [ax1,ax2]:
    ax.legend(handles=[l1,l2,l3,l4,l5],handlelength=1.5,framealpha=1,labelspacing=.3,borderpad=0.5,
                    handletextpad=0.5,loc='upper right',frameon=False,markerfirst=False)
    ax.get_legend()._legend_box.align = 'right'
    ax.set_yticks(np.arange(0,2.1,1))
    ax.set_ylim(-.14,2.5)
    ax.set_xlim(0,180)
    ax.set_xticks(np.arange(0,181,30))
    ax.set_ylabel(r'Probability, $P(\theta)$')
ax1.annotate(r'a',xy=(0.05,.88), fontsize=22, xycoords='axes fraction')
ax2.annotate(r'b',xy=(0.05,.88), fontsize=22, xycoords='axes fraction') 

#ax2.vlines(x=63,ymin=0,ymax=3)
#ax2.vlines(x=45.5,ymin=0,ymax=1.5)
#ax2.vlines(x=80.5,ymin=0,ymax=1.5)
    
plt.tight_layout(h_pad=0.35)
plt.savefig('figs/figS8.pdf')

In [ ]:
plt.rcParams.update({'figure.figsize': [10, 4], 'figure.dpi':70})
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=False)

func = lambda x,a : (a*a + 1)/(a*a*np.exp(-np.pi/a)+a*a)*np.exp(-x/a)
x = np.linspace(0,np.pi,100)
#ax.fill_between(x/np.pi*180,func(x,50/180*np.pi),func(x,40/180*np.pi),alpha=0.2,color='k',lw=0)
ax1.plot(x/np.pi*180,func(x,40/180*np.pi),color='k',ls=':')
ax1.plot(x/np.pi*180,func(x,50/180*np.pi),color='k',ls='--')
ax2.plot(x/np.pi*180,func(x,40/180*np.pi),color='k',ls=':')
ax2.plot(x/np.pi*180,func(x,50/180*np.pi),color='k',ls='--')

method = 'gds'
pltDang(ax1,['water'],method,color=7,l='0')
pltDang(ax1,[150,300,450,600,800],method)

method = 'wcis'
pltDang(ax2,['water'],method,color=7,l='0')
pltDang(ax2,[150,300,450,600,800],method)

ax1.set_xlabel(r'Dangling OH Tilt Angle, $\psi$ (˚)')
ax2.set_xlabel(r'Dangling OH Tilt Angle, $\psi$ (˚)')
ax1.set_yticks(np.arange(0,3.1,1))
ax2.set_yticks(np.arange(0,3.1,1))
ax1.set_ylim(0,3)
ax2.set_ylim(0,3)
ax1.set_xlim(0,180)
ax1.set_xticks(np.arange(0,181,30))
ax1.set_ylabel(r'Probability, $P(\psi)$')
ax2.set_ylabel(r'Probability, $P(\psi)$')
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
ax1.legend(frameon=False,handlelength=1.2,markerfirst=False, title='$c_s$ (M)', loc='center right', ncol=2)
ax2.legend(frameon=False,handlelength=1.2,markerfirst=False, title='$c_s$ (M)', loc='center right', ncol=2)
ax1.annotate(r'a',xy=(0.88,.88), fontsize=22, xycoords='axes fraction')
ax2.annotate(r'b',xy=(0.88,.88), fontsize=22, xycoords='axes fraction') 
plt.tight_layout(w_pad=0.5)
plt.savefig('figs/figS9.pdf')
plt.show()

In [ ]:
def pltDang(ax,Ns,method,color=None,l=''):
    for i,N in enumerate(Ns):
        if color==None:
            color=i
        theta = pd.read_pickle(method+'/0/{0}.p'.format(N))['theta']/180*np.pi
        pu = np.zeros(theta.size)
        pl = np.zeros(theta.size)
        for n in range(10):
            params = pd.read_pickle(method+'/{0}/{1}.p'.format(n,N))
            pu += params['data']['O']['upper']['theta']
            pl += params['data']['O']['lower']['theta']
        pu = pu / np.trapz(pu,theta) / np.sin(theta)
        pl = pl / np.trapz(pl,theta) / np.sin(theta)
        p = .5*(pu+pl)
        dp = .5*np.abs(pu-pl)
        conc = 0
        
        func = lambda x,a : np.log(a*a + 1) - np.log(a*a*(np.exp(-np.pi/a)+1)) - x / a  
        
        popt, pcov = curve_fit(func, theta, np.log(p))
        
        ax.fill_between(theta*180/np.pi,p+dp,p-dp,color=plt.cm.tab10(color),alpha=.2)
        
        ax.plot(theta*180/np.pi,np.exp(func(theta,popt[0])),lw=1,color=plt.cm.tab10(color),ls=':')
        
        if 'C2' in params['data']:
            _, __, ___, ____, conc = avgConc(N,range(10),'C2','gds')
            ax.plot(theta*180/np.pi,p,c=plt.cm.tab10(color),lw=2,
                    label=l+r'$c_s={:.1f}$ M, $\psi_E={:.0f}˚$'.format(conc,popt[0]*180/np.pi))
        else:
            ax.plot(theta*180/np.pi,p,c=plt.cm.tab10(color),lw=2,
                    label=l+r'$\psi_E={:.0f}˚$'.format(popt[0]*180/np.pi))
        color=None

In [ ]:
plt.rcParams.update({'figure.figsize': [6.5, 5.5], 'figure.dpi':70})
ax = plt.subplot()
method = 'wcis'
pltDang(ax,['water'],method,color=7,l='0.0')
pltDang(ax,[150,300,450,600,800],method)

ax.set_xlabel(r'Dangling OH Tilt Angle, $\psi$ (˚)')
ax1.set_yticks(np.arange(0,4.1,1))
ax1.set_ylim(0,4.5)
#ax.set_xlim(0,180)
#ax.set_xticks(np.arange(0,181,30))
ax.set_ylabel(r'Probability $\times$ 100')
plt.legend(frameon=False,handlelength=1.2,markerfirst=False)
plt.tight_layout()
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize':[6.5, 3.5],'legend.frameon':False,'figure.dpi':70})
fig, ax1 = plt.subplots()

method = 'wcis'

ax1.tick_params(axis='y',colors=plt.cm.tab10(0))
ax1.set_ylabel(r'$\langle \psi \rangle$ (˚)',color=plt.cm.tab10(0))

pltAveragePsi(ax1,['water',150,300,450,600,800],method)
#pltAveragePsi(ax1,['water',150,300,450,600,800],'gds')

ax2 = ax1.twinx() # creates a new subplot identical to x1, with invisible x-axis and y-axis at right
ax2.tick_params(axis='y',colors=plt.cm.tab10(3))
ax2.set_ylabel(r'Fraction (%)',color=plt.cm.tab10(3))

#pltFrac(ax2,['water',150,300,450,600,800],'gds',color=plt.cm.tab10(3))
pltFrac(ax2,['water',150,300,450,600,800],method,color=plt.cm.tab10(3))

ax1.set_xlabel(r'Salt Concentration, $c_s$  (M)')

ax1.set_ylim(50,60); ax2.set_ylim(20,50)
ax1.set_xlim(-.5,5.5)
ax1.set_xticks(np.arange(0,5.1,1))
ax1.set_xticklabels(['{:1g}'.format(i) for i in np.arange(0,5.1,1)])

fig.tight_layout()
plt.savefig('figs/fig5.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize':[6.5, 3.5],'legend.frameon':False,'figure.dpi':70})
fig, ax1 = plt.subplots()

method = 'gds'

ax1.tick_params(axis='y',colors=plt.cm.tab10(0))
ax1.set_ylabel(r'$\langle \psi \rangle$ (˚)',color=plt.cm.tab10(0))

#pltAveragePsi(ax1,['water',150,300,450,600,800],method)
pltAveragePsi(ax1,['water',150,300,450,600,800],'gds')

ax2 = ax1.twinx() # creates a new subplot identical to x1, with invisible x-axis and y-axis at right
ax2.tick_params(axis='y',colors=plt.cm.tab10(3))
ax2.set_ylabel(r'Fraction (%)',color=plt.cm.tab10(3))

pltFrac(ax2,['water',150,300,450,600,800],'gds',color=plt.cm.tab10(3))
#pltFrac(ax2,['water',150,300,450,800],method,color=plt.cm.tab10(3))

ax1.set_xlabel(r'Salt Concentration, $c_s$  (M)')

ax1.set_ylim(50,60); ax2.set_ylim(20,50)
ax1.set_xlim(-.5,5.5)
ax1.set_xticks(np.arange(0,5.1,1))
ax1.set_xticklabels(['{:1g}'.format(i) for i in np.arange(0,5.1,1)])

fig.tight_layout()
plt.savefig('figs/figS10.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize':[6.5, 3.5],'legend.frameon':False,'figure.dpi':70})
fig, ax1 = plt.subplots()

method = 'wcis'

ax1.tick_params(axis='y',colors=plt.cm.tab10(0))
ax1.set_ylabel(r'$\langle \psi \rangle$ (˚)',color=plt.cm.tab10(0))

pltAveragePsi(ax1,['water',150,300,450,600,800],method,color=plt.cm.tab10(0))
pltAveragePsi(ax1,['water',150,300,450,600,800],'gds',color=plt.cm.tab10(1))

ax2 = ax1.twinx() # creates a new subplot identical to x1, with invisible x-axis and y-axis at right
ax2.tick_params(axis='y',colors=plt.cm.tab10(3))
ax2.set_ylabel(r'Fraction (%)',color=plt.cm.tab10(3))

pltNumDang(ax2,['water',150,300,450,600,800],'gds',color=plt.cm.tab10(2))
pltNumDang(ax2,['water',150,300,450,800],method,color=plt.cm.tab10(3))

ax1.set_xlabel(r'Salt Concentration, $c_s$  (M)')

ax1.set_ylim(50,60); #ax2.set_ylim(20,50)
ax1.set_xlim(-.5,5.5)
ax1.set_xticks(np.arange(0,5.1,1))
ax1.set_xticklabels(['{:1g}'.format(i) for i in np.arange(0,5.1,1)])

fig.tight_layout()
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [10, 10], 'figure.dpi':70})
plt.rcParams.update({'font.size': 15,'xtick.major.pad':3,'ytick.major.pad':3,'figure.dpi':150,
                     'xtick.major.size':6,'ytick.major.size':6,'legend.fontsize':15,
                     'xtick.direction':'out','ytick.direction':'out','axes.labelsize':15,
                     'axes.linewidth':1.2,'xtick.labelsize':15,'ytick.labelsize':15,
                     'xtick.major.width':1.2, 'ytick.major.width':1.2})
fig, ((ax1, ax2),(ax3, ax4),(ax5, ax6)) = plt.subplots(3, 2, sharex=False, sharey=False)

method = 'wcis'
surfconc, excess = pltConc(ax1,'C2',[447,596,894,1192],method)
np.savetxt(method+'/surfconc_air.dat',surfconc)
pltConc(ax3,'NA',[447,596,894,1192],method)
pltConc(ax5,'O',[447,596,894,1192],method)
#np.savetxt('willchan/excess_air.dat',excess)
surfconc, excess = pltConc(ax2,'C2',[150,300,450,600,800],method)
np.savetxt(method+'/surfconc_mono.dat',surfconc)
#np.savetxt('willchan/excess_mono.dat',excess)
pltConc(ax4,'NA',[150,300,450,600,800],method)
pltConc(ax6,'O',[150,300,450,600,800],method)

for ax in fig.axes[:2]:
    ax.set_xlim(-.5,2.5)
    ax.set_ylim(-.2,14)
    ax.set_ylabel('$c_{C}(z)$  (M)')  
    ax.set_xticks(np.arange(0,2.6,1))
    ax.set_xticklabels(np.tile('',3))
    ax.set_yticks(np.arange(0,14,3))
    ax.legend(loc='upper right',handletextpad=0.5,markerfirst=False,fontsize=15,
              frameon=False,handlelength=1,title='$c_s$ (M)',ncol=2,)
    
for ax in fig.axes[2:4]:
    ax.set_xlim(-.5,2.5)
    ax.set_ylim(-.2,10)
    ax.set_ylabel('$c_{Na}(z)$  (M)')  
    ax.set_xticks(np.arange(0,2.6,1))
    ax.set_xticklabels(np.tile('',3))
    ax.legend(loc='upper right',handletextpad=0.5,markerfirst=False,fontsize=15,
              frameon=False,handlelength=1,title='$c_s$ (M)',ncol=2,)
    
for ax in fig.axes[4:]:
    ax.set_xlim(-.5,2.5)
    ax.set_ylim(-1,90)
    ax.set_ylabel('$c_{O}(z)$  (M)')  
    ax.set_xlabel(r'Distance from Interface, $z$ (nm)') 
    ax.set_xticks(np.arange(0,2.6,1))
    ax.set_xticklabels(['{:1g}'.format(i) for i in np.arange(0,2.6,1)])
    ax.legend(loc='lower right',handletextpad=0.5,markerfirst=False,fontsize=15,
              frameon=False,handlelength=1,title='$c_w$ (M)',ncol=2,)

    
ax1.annotate(r'a',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax2.annotate(r'b',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax3.annotate(r'c',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax4.annotate(r'd',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax5.annotate(r'e',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax6.annotate(r'f',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
    
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
ax4.yaxis.tick_right()
ax4.yaxis.set_label_position("right")
ax6.yaxis.tick_right()
ax6.yaxis.set_label_position("right")

#ax2.vlines(x=0.4,ymin=-.4,ymax=8)
#ax2.vlines(x=0.4,ymin=-.4,ymax=8)
#ax6.vlines(x=0.31,ymin=-.4,ymax=80)

plt.tight_layout(w_pad=.5,h_pad=.5)
plt.savefig('figs/figS6.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [10, 10], 'figure.dpi':70})
plt.rcParams.update({'font.size': 15,'xtick.major.pad':3,'ytick.major.pad':3,'figure.dpi':150,
                     'xtick.major.size':6,'ytick.major.size':6,'legend.fontsize':15,
                     'xtick.direction':'out','ytick.direction':'out','axes.labelsize':15,
                     'axes.linewidth':1.2,'xtick.labelsize':15,'ytick.labelsize':15,
                     'xtick.major.width':1.2, 'ytick.major.width':1.2})
fig, ((ax1, ax2),(ax3, ax4),(ax5, ax6)) = plt.subplots(3, 2, sharex=False, sharey=False)

method = 'gds'
surfconc, excess = pltConc(ax1,'C2',[447,596,894,1192],method)
#np.savetxt(method+'/surfconc_air.dat',surfconc)
np.savetxt(method+'/excess_air.dat',excess)
surfconc, excess = pltConc(ax2,'C2',[150,300,450,600,800],method)
#np.savetxt(method+'/surfconc_mono.dat',surfconc)
np.savetxt(method+'/excess_mono.dat',excess)

pltConc(ax3,'NA',[447,596,894,1192],method)
pltConc(ax4,'NA',[150,300,450,600,800],method)

pltConc(ax5,'O',[447,596,894,1192],method)
pltConc(ax6,'O',[150,300,450,600,800],method)

for ax in fig.axes[:2]:
    ax.set_xlim(-.5,2.5)
    ax.set_ylim(-.2,14)
    ax.set_ylabel('$c_{C}(z)$  (M)')  
    ax.set_xticks(np.arange(0,2.6,1))
    ax.set_xticklabels(np.tile('',3))
    ax.set_yticks(np.arange(0,14,3))
    ax.legend(loc='upper right',handletextpad=0.5,markerfirst=False,fontsize=15,
              frameon=False,handlelength=1,title='$c_s$ (M)',ncol=2,)
    
for ax in fig.axes[2:4]:
    ax.set_xlim(-.5,2.5)
    ax.set_ylim(-.2,10)
    ax.set_ylabel('$c_{Na}(z)$  (M)')  
    ax.set_xticks(np.arange(0,2.6,1))
    ax.set_xticklabels(np.tile('',3))
    ax.legend(loc='upper right',handletextpad=0.5,markerfirst=False,fontsize=15,
              frameon=False,handlelength=1,title='$c_s$ (M)',ncol=2,)
    
for ax in fig.axes[4:]:
    ax.set_xlim(-.5,2.5)
    ax.set_ylim(-1,90)
    ax.set_ylabel('$c_{O}(z)$  (M)')  
    ax.set_xlabel(r'Distance from Interface, $z$ (nm)') 
    ax.set_xticks(np.arange(0,2.6,1))
    ax.set_xticklabels(['{:1g}'.format(i) for i in np.arange(0,2.6,1)])
    ax.legend(loc='lower right',handletextpad=0.5,markerfirst=False,fontsize=15,
              frameon=False,handlelength=1,title='$c_w$ (M)',ncol=2,)

    
ax1.annotate(r'a',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax2.annotate(r'b',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax3.annotate(r'c',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax4.annotate(r'd',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax5.annotate(r'e',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
ax6.annotate(r'f',xy=(0.05,.87), fontsize=22, xycoords='axes fraction')
    
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
ax4.yaxis.tick_right()
ax4.yaxis.set_label_position("right")
ax6.yaxis.tick_right()
ax6.yaxis.set_label_position("right")

#ax2.vlines(x=0.4,ymin=-.4,ymax=8)
#ax2.vlines(x=0.4,ymin=-.4,ymax=8)
#ax6.vlines(x=0.3,ymin=-.4,ymax=80)

plt.tight_layout(w_pad=.5,h_pad=.5)
plt.savefig('figs/figS7.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [6.5, 4], 'figure.dpi':70})
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False)

conc,n,En = np.loadtxt('gds/excess_mono.dat',unpack=True)

ax1.errorbar(conc,n,En,marker='o',color=plt.cm.tab10(0),
        markeredgecolor=None,markersize=10,elinewidth=2,capsize=4,
        lw=0,capthick=2.,alpha=1)

conc,n,En = np.loadtxt('gds/excess_air.dat',unpack=True)

ax1.errorbar(conc,n,En,marker='o',color=plt.cm.tab10(3),
        markeredgecolor=None,markersize=10,elinewidth=2,capsize=4,
        lw=0,capthick=2.,alpha=1)

conc,n,En = np.loadtxt('wcis/surfconc_mono.dat',unpack=True)

ax2.errorbar(conc,n,En,marker='o',color=plt.cm.tab10(0),
        markeredgecolor=None,markersize=10,elinewidth=2,capsize=4,
        lw=0,capthick=2.,alpha=1)

conc,n,En = np.loadtxt('wcis/surfconc_air.dat',unpack=True)

ax2.errorbar(conc,n,En,marker='o',color=plt.cm.tab10(3),
        markeredgecolor=None,markersize=10,elinewidth=2,capsize=4,
        lw=0,capthick=2.,alpha=1)

ax1.annotate(r'a',xy=(0.05,.89), fontsize=22, xycoords='axes fraction')
ax2.annotate(r'b',xy=(0.05,.89), fontsize=22, xycoords='axes fraction')
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")


ax2.set_xlabel(r'$c_s$  (M)')
ax1.set_xlabel(r'$c_s$  (M)')

ax2.set_ylabel(r'Surface Concentration, $\sigma$  (nm$^{-2})$')
ax1.set_ylabel(r'Surface Excess, $\Gamma$  (nm$^{-2})$')

ax2.set_xlim(.2,5.2); ax1.set_xlim(.2,5.2)
ax2.set_ylim(0,1.4); ax1.set_ylim(-.4,.1)

ax2.set_xticks(np.arange(1,5.1,1)); ax1.set_xticks(np.arange(1,5.1,1))
ax2.set_yticks(np.arange(0,1.4,.3))
ax2.set_yticklabels(['{:1g}'.format(i) for i in np.arange(0,1.4,.3)])
ax1.set_yticks(np.arange(-.4,.1,.1))
ax1.set_yticklabels(['{:1.1f}'.format(i) for i in np.arange(-.4,.1,.1)])

fig.tight_layout(w_pad=1, h_pad=0.2)
fig.savefig('figs/fig7.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [6.5, 4]})
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False)

#######
method = 'wcis'
pltCosine(ax1,'O',[150,300,450,600,800],method)
wsurfconc = pltCosine(ax2,'C2',[150,300,450,600,800],method)
np.savetxt(method+'/wsurfconc_mono.dat',wsurfconc)

ax1.set_xlim(-.4,2)
ax2.set_xlim(-.2,1.5)

#######

ax = plt.axes([.38, .42, .1, .1])
img=mpl.image.imread('aux/spce.png')
ax.imshow(img,interpolation='none')
ax.axis('off')

ax = plt.axes([.7, .40, .12, .12])
img=mpl.image.imread('aux/scn.png')
ax.imshow(img,interpolation='none')
ax.axis('off')

ax1.annotate(r'a',xy=(0.05,.89), fontsize=22, xycoords='axes fraction')
ax2.annotate(r'b',xy=(0.05,.89), fontsize=22, xycoords='axes fraction')
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")


ax1.set_xlabel(r'$z$  (nm)')
ax2.set_xlabel(r'$z$  (nm)')

ax1.set_ylabel(r'$c_{O}(z) \langle \cos\phi \rangle(z)$  (M)')
ax2.set_ylabel(r'$c_{C}(z) \langle \cos\theta \rangle(z)$  (M)')

ax1.set_xticks(np.arange(0,2.1,.5)); ax2.set_xticks(np.arange(0,1.6,.5))
ax1.set_xticklabels(['{:1g}'.format(i) for i in np.arange(0,2.1,.5)])
ax2.set_xticklabels(['{:1g}'.format(i) for i in np.arange(0,1.6,.5)])

ax1.set_yticks(np.arange(-2.5,8.1,1.5)); ax2.set_yticks(np.arange(-2.5,8.1,1.5))

ax1.legend(loc='upper right',handletextpad=0.5,markerfirst=False,fontsize=12,
              frameon=False,handlelength=1,title='$c_w$ (M)')
ax2.legend(loc='upper right',handletextpad=0.5,markerfirst=False,fontsize=12,
              frameon=False,handlelength=1,title='$c_s$ (M)')

ax1.set_ylim(-2.5,8); ax2.set_ylim(-2.5,8)

#ax2.vlines(x=0.2,ymin=-.4,ymax=3)
#ax2.vlines(x=0.75,ymin=-.4,ymax=3)

fig.tight_layout(w_pad=.5, h_pad=0.2)
fig.savefig('figs/fig6.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize':[6.5, 3.5],'legend.frameon':False,'figure.dpi':70})
fig, ax1 = plt.subplots()

ax1.set_xlabel(r'Mole Fraction, $x$')
ax1.tick_params(axis='y',colors=plt.cm.tab10(0))
ax1.set_ylabel(r'$\sigma$ / $\sigma(c_s=4.7 \mathrm{M})$',color=plt.cm.tab10(0))

conc,z,Ez = np.loadtxt('wcis/surfconc_mono.dat',unpack=True)
z/=z[-1]

ax1.errorbar(conc,z,Ez,marker='o',color=plt.cm.tab10(0),
        markeredgecolor=None,markersize=10,elinewidth=2,capsize=4,
        lw=0,capthick=2.,alpha=1)


ax2 = ax1.twinx() # creates a new subplot identical to x1, with invisible x-axis and y-axis at right
ax2.tick_params(axis='y',colors=plt.cm.tab10(3))
ax2.set_ylabel(r'$\sigma^{\theta}$ / $\sigma^{\theta}(c_s=4.7 \mathrm{M})$',color=plt.cm.tab10(3))

conc,z,Ez = np.loadtxt('wcis/wsurfconc_mono.dat',unpack=True)
z/=z[-1]

ax2.errorbar(conc,z,Ez,marker='s',color=plt.cm.tab10(3),
        markeredgecolor=None,markersize=6,elinewidth=2,capsize=2,
        lw=0,capthick=2.,alpha=1)

#ax1.plot(conc,k,'ro')
#print(conc/k)

f = lambda x, *p: x*1.27/(x+55.5*np.exp(p[0]/8.3145/.298))
popt,pcov = curve_fit(f,conc,z,p0=[.1],sigma=Ez)

print(popt[0],np.sqrt(np.diag(pcov))[0])

#x,y = np.loadtxt('data/experimental.txt',unpack=True)
#popt,pcov = curve_fit(f,x,y,p0=[.1])

#print(popt[0],np.sqrt(np.diag(pcov))[0])

x = np.arange(0,5.5,.01)
ax1.plot(x,f(x,popt[0]),color=plt.cm.tab10(0))
ax1.plot(x,f(x,-9.8),'k--')

ax1.set_xlabel(r'Salt Concentration, $c_s$  (M)')

ax1.set_ylim(0,1.2); ax2.set_ylim(0,1.2)
ax1.set_xlim(0,5.5)
ax1.set_xticks(np.arange(0,5.1,1))
ax1.set_xticklabels(['{:1g}'.format(i) for i in np.arange(0,5.1,1)])

fig.tight_layout()
plt.savefig('figs/fig8.pdf')
plt.show()

In [ ]:
plt.rcParams.update({'figure.figsize': [10, 4], 'figure.dpi':70})
fig, (ax1,ax2) = plt.subplots(1, 2, sharex=False, sharey=False)
plt.rcParams.update({'font.size': 15,'xtick.major.pad':3,'ytick.major.pad':3,'figure.dpi':150,
                     'xtick.major.size':6,'ytick.major.size':6,'legend.fontsize':15,
                     'xtick.direction':'out','ytick.direction':'out','axes.labelsize':15,
                     'axes.linewidth':1.2,'xtick.labelsize':15,'ytick.labelsize':15,
                     'xtick.major.width':1.2, 'ytick.major.width':1.2})
#######
method = 'wcis'
pltCosineNorm(ax1,'O',[150,300,450,600,800],method)
pltCosineNorm(ax2,'C2',[150,300,450,600,800],method)
#######

for ax in fig.axes:
    ax.set_xlabel(r'$z$  (nm)')
    ax.set_xticks(np.arange(0,2.1,.5))
    ax.set_xticklabels(['{:1g}'.format(i) for i in np.arange(0,2.1,.5)])
    

ax1.set_ylabel(r'$\langle \cos\phi \rangle(z)$')
ax2.set_ylabel(r'$\langle \cos\theta \rangle(z)$')

ax1.annotate(r'a',xy=(0.05,.89), fontsize=22, xycoords='axes fraction')
ax2.annotate(r'b',xy=(0.05,.89), fontsize=22, xycoords='axes fraction')
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")



ax1.set_yticks(np.arange(-.05,.21,.05))
ax1.set_yticklabels(['{:1g}'.format(i) for i in np.arange(-.05,.21,.05)])

ax2.set_yticks(np.arange(0,.9,.2))
ax2.set_yticklabels(['{:1g}'.format(i) for i in np.arange(0,.9,.2)])

#ax1.set_yticks(np.arange(-1,3.1,1)); ax2.set_yticks(np.arange(0,3.1,1))

ax1.set_xlim(-.1,2)
ax2.set_xlim(0,2)
ax1.set_ylim(-.05,.2)
ax2.set_ylim(-.15,.8)

ax1.legend(loc='upper right',handletextpad=0.5,markerfirst=False,fontsize=15,
              frameon=False,handlelength=1,title='$c_w$ (M)')
ax2.legend(loc='upper right',handletextpad=0.5,markerfirst=False,fontsize=15,
              frameon=False,handlelength=1,title='$c_s$ (M)')

for ax in [ax1,ax2]:
    #ax.vlines(x=.1,ymin=-1,ymax=8,lw=.5,linestyle=':')
    #ax.vlines(x=.3,ymin=-1,ymax=8,lw=.5,linestyle=':')
    #ax.vlines(x=.5,ymin=-1,ymax=8,lw=.5,linestyle=':')
    #ax.vlines(x=.7,ymin=-1,ymax=8,lw=.5,linestyle=':')
    #ax.vlines(x=.9,ymin=-1,ymax=8,lw=.5,linestyle=':')
    ax.axvspan(-.1, .1, color=plt.cm.tab10(2), alpha=0.1, ec=None)
    ax.axvspan(.1, .3, color=plt.cm.tab10(3), alpha=0.1, ec=None)
    ax.axvspan(.3, .5, color=plt.cm.tab10(4), alpha=0.1, ec=None)
    ax.axvspan(.5, .7, color=plt.cm.tab10(8), alpha=0.1, ec=None)
    ax.axvspan(.7, .9, color=plt.cm.tab10(7), alpha=0.1, ec=None)

fig.tight_layout(w_pad=.5, h_pad=0.2)
fig.savefig('figs/figS11.pdf')
plt.show()